In [2]:
from langchain_groq import ChatGroq
from langchain_core.tools import Tool
from langchain_google_community import GoogleSearchAPIWrapper
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.agents import create_react_agent,AgentExecutor
from langchain_core.prompts import PromptTemplate
from langchain.tools.retriever import create_retriever_tool
from langchain.memory import ConversationBufferMemory

In [24]:
GROQ_API_KEY='gsk_MJkiiYtggBpMNHFvfwefFf8ayiOm3YeP2321SFOn'
GOOGLE_API_KEY='AIzaBGhmmIUIOPiytgeewfNGhjopbvftyP1BS-HZiyoY'
SEARCH_ENGINE_ID='437hjk789mdhd89wtf94ab9'

In [25]:
llm=ChatGroq(model='llama-3.1-70b-versatile',api_key=GROQ_API_KEY,temperature=1e-08)

In [26]:
llm.invoke("explain about generative AI")

AIMessage(content='**Generative AI: An Overview**\n\nGenerative AI, also known as Generative Models or Generative Machine Learning, is a type of artificial intelligence that uses algorithms to generate new, synthetic data that resembles existing data. This technology has gained significant attention in recent years due to its potential to revolutionize various industries, including art, music, writing, and more.\n\n**How Generative AI Works**\n\nGenerative AI models are trained on large datasets of existing data, such as images, text, or audio. The model learns patterns, relationships, and structures within the data, allowing it to generate new data that is similar in style and content. The process involves the following steps:\n\n1. **Data Collection**: Gathering a large dataset of existing data, such as images, text, or audio.\n2. **Model Training**: Training a generative model on the collected data, using algorithms such as Generative Adversarial Networks (GANs), Variational Autoenc

In [27]:
llm.invoke("todays petrol price in bangalore")

AIMessage(content='I\'m not able to provide real-time information. However, I can suggest some ways for you to find the current petrol price in Bangalore.\n\n1. **IOCL Website**: You can visit the Indian Oil Corporation Limited (IOCL) website ([www.iocl.com](http://www.iocl.com)) and click on the "Petrol/Diesel Price" link to find the current prices in Bangalore.\n2. **Fuel Price Websites**: Websites like [www.fuelprice.io](http://www.fuelprice.io), [www.fuelpriceindia.com](http://www.fuelpriceindia.com), or [www.petrolprice.com](http://www.petrolprice.com) provide current fuel prices in various cities, including Bangalore.\n3. **Mobile Apps**: You can download mobile apps like IOCL\'s "Fuel@IOC" or "Petrol Price India" to find the current petrol prices in Bangalore.\n4. **Google Search**: Simply type "current petrol price in Bangalore" or "petrol price in Bangalore today" in Google to find the latest prices.\n\nPlease note that fuel prices are subject to change and may vary depending 

In [28]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [29]:
#Tool1 wikipedia
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
tool_wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [30]:
#Tool2 Google search
search = GoogleSearchAPIWrapper(google_api_key=GOOGLE_API_KEY,google_cse_id=SEARCH_ENGINE_ID)
tool_google = Tool(
                name="google_search",
                description="Search Google for recent results.",
                func=search.run,
            )

In [31]:
tools=[tool_google,tool_wiki]

In [32]:
#prompt template
prompt=PromptTemplate(
            input_variables=['chat_history', 'agent_scratchpad', 'input', 'tool_names', 'tools'],
            metadata={
                'lc_hub_owner': 'hwchase17',
                'lc_hub_repo': 'react',
                'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'
            },
            template=(
                'You are an intelligent assistant that can answer questions and perform actions using the tools provided.\n'
                'You also have access to the conversation history, which you should use to provide more relevant and context-aware responses.\n\n'
                'Here are the available tools:\n\n{tools}\n\n'
                'Here is the chat history so far:\n\n{chat_history}\n\n'
                'When responding, take the chat history into account.\n'
                'Use the following format to answer the questions:\n\n'
                'Question: the input question you must answer\n'
                'Thought: you should always think about what to do based on the chat history and tools\n'
                'Action: the action to take, should be one of [{tool_names}]\n'
                'Action Input: the input to the action\n'
                'Observation: the result of the action\n'
                '... (this Thought/Action/Action Input/Observation can repeat N times)\n'
                'Thought: I now know the final answer\n'
                'Final Answer: the final answer to the original input question, considering the chat history\n\n'
                'Begin!\n\n'
                'Question: {input}\n'
                'Thought: {agent_scratchpad}'
            )
        )

In [33]:
agent = create_react_agent(
            llm=llm,
            tools=tools,
            prompt=prompt,
            stop_sequence=True,
        )

In [34]:
agent_executor = AgentExecutor(agent=agent, tools=tools,memory=memory,handle_parsing_errors=True)

In [35]:
response=agent_executor.invoke({"input": "todays petrol price in bangalore"})

In [36]:
response

{'input': 'todays petrol price in bangalore',
 'chat_history': [HumanMessage(content='todays petrol price in bangalore', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The current petrol price in Bangalore as of 14 October 2024 is Rs. 103.57 per litre.', additional_kwargs={}, response_metadata={})],
 'output': 'The current petrol price in Bangalore as of 14 October 2024 is Rs. 103.57 per litre.'}

In [37]:
print(response.get("chat_history"))

[HumanMessage(content='todays petrol price in bangalore', additional_kwargs={}, response_metadata={}), AIMessage(content='The current petrol price in Bangalore as of 14 October 2024 is Rs. 103.57 per litre.', additional_kwargs={}, response_metadata={})]


In [38]:
response=agent_executor.invoke({"input": "list out the movies running in same place"})

In [39]:
print(response.get("chat_history"))

[HumanMessage(content='todays petrol price in bangalore', additional_kwargs={}, response_metadata={}), AIMessage(content='The current petrol price in Bangalore as of 14 October 2024 is Rs. 103.57 per litre.', additional_kwargs={}, response_metadata={}), HumanMessage(content='list out the movies running in same place', additional_kwargs={}, response_metadata={}), AIMessage(content='Based on the available information, here are some movies that are currently running in theaters in Bangalore:\n\n* Vettaiyan (Tamil)\n* Vicky Vidya Ka Woh Video (Hindi)\n* Jigra (Hindi)\n\nPlease note that this list may not be comprehensive or up-to-date, and I recommend checking with local theaters or online ticketing websites for the most current information.', additional_kwargs={}, response_metadata={})]
